In [ ]:
import numpy as np
import pydicom
import nibabel as nib
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.metrics import roc_curve, auc
from time import perf_counter
from scipy.stats import wilcoxon, pearsonr
from scipy.ndimage import label
from matplotlib.collections import LineCollection

In [ ]:
#function for reading dicom images
#input: path to the folder with dicom slices
#output: 3D image array, a vector specifying the voxel size, another vector specifying the image position w.r.t. the patient
def readDcmImage(pathToDcmFolder):
    files=os.listdir(pathToDcmFolder)
    dcmFiles=[]
    for i in range(len(files)):
        if files[i][-4:]=='.dcm':
            dcmFiles.append(files[i])
    file='{}/{}'.format(pathToDcmFolder,dcmFiles[0])
    ds=pydicom.dcmread(file)
    iop=ds.ImageOrientationPatient
    locationIndex=np.argmin(np.array(iop[0:3])+np.array(iop[3:6]))
    sliceList=[]
    locations=[]
    for i in range(len(dcmFiles)):
        file='{}/{}'.format(pathToDcmFolder,dcmFiles[i])
        ds=pydicom.dcmread(file)
        locations.append(ds.ImagePositionPatient[locationIndex])
        if 'RescaleSlope' in ds and 'RescaleIntercept' in ds:
            sliceList.append(ds.pixel_array*ds.RescaleSlope+ ds.RescaleIntercept)
        else:
            sliceList.append(ds.pixel_array)
    uniqueLocations=np.sort(np.unique(np.array(locations)))
    voxelSize=np.array([ds.PixelSpacing[0],ds.PixelSpacing[1], uniqueLocations[1]-uniqueLocations[0]])
    merged=dict(zip(locations,sliceList))
    orderedSliceList=[merged[loc] for loc in sorted(locations)]
    img3d=np.moveaxis(np.array(orderedSliceList),0,2)
    img3d=np.moveaxis(img3d,0,1)
    return(img3d,voxelSize,iop)

#function for preprocessing 2D image slices (performs resizing, normalization, and converting into float16)
def preprocessImgSlice(img,img_height):
    if np.max(img)>np.min(img):
        img=cv2.resize(img,(img_height,img_height))
        img=(img-np.min(img))/(np.max(img)-np.min(img))
    else:
        img=np.zeros((img_height,img_height))
    return img.astype('float16')

#function to perform the five-fold cross-validation
#inputs: lists of image slices, their ground-truth labels, and the patient indexes, and a number k=0,1,...,4 specifying the test set fold
#output: training and test set data and the corresponding patient indexes
def fiveFoldCrossValidation(slices,labels,patientIndexes,k):
    x_train=[]
    y_train=[]
    x_test=[]
    y_test=[]
    patientIndexes_test=[]
    patientIndexes_train=[]
    for i in range(len(patientIndexes)):
        if patientIndexes[i] % 5==k:
            x_test.append(slices[i])
            y_test.append(labels[i])
            patientIndexes_test.append(patientIndexes[i])
        else:
            x_train.append(slices[i])
            y_train.append(labels[i])
            patientIndexes_train.append(patientIndexes[i])
    x_train=np.array(x_train)
    y_train=np.array(y_train)    
    x_test=np.array(x_test)
    y_test=np.array(y_test)
    return(x_train,y_train,x_test,y_test,patientIndexes_train,patientIndexes_test)

#function for training the encoderCNN for classification and then predicting the test and train set
def predict(x_train,y_train,x_test,numberOfEpochs):
    
    img_height=x_train[0].shape[0]

    model = keras.Sequential([keras.layers.Conv2D(16, 3, activation='relu', input_shape=(img_height,img_height,1)),
                        keras.layers.Conv2D(16, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(32, 3, activation='relu'),
                        keras.layers.Conv2D(32, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(64, 3, activation='relu'),
                        keras.layers.Conv2D(64, 3, activation='relu'),
                        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                        keras.layers.Conv2D(128, 3, activation='relu'),
                        keras.layers.Conv2D(128, 3, activation='relu'),
                        keras.layers.MaxPooling2D(strides=(2, 2)),
                        keras.layers.Flatten(),
                        keras.layers.Dense(128, activation='relu'),
                        keras.layers.Dense(64, activation='relu'),
                        keras.layers.Dense(32, activation='relu'),
                        keras.layers.Dense(1, activation='sigmoid')
    ]
    )

    model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-3),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    history=model.fit(x=x_train,y=y_train,epochs=numberOfEpochs,shuffle=True)
    predictions=model.predict(x_test)[:,0]
    trainPreds=model.predict(x_train)[:,0]
    plt.plot(range(len(history.history['loss'])),history.history['loss'],color='blue')

    return([predictions,trainPreds])

#function for post-processing predictions by taking means of five consecutive predictions
def postProcess_new(predictions,patientIndexes_test):

    predictions_1=[]
    for i in range(len(predictions)):
        current=predictions[i]
        if i>0:
            if patientIndexes_test[i-1]==patientIndexes_test[i]:
                prev=predictions[i-1]
            else:
                prev=0
            if i>1:
                if patientIndexes_test[i-2]==patientIndexes_test[i]:
                    prev1=predictions[i-2]
                else:
                    prev1=0
            else:
                    prev1=0
        else:
            prev=0
            prev1=0
        if i<len(predictions)-1:
            if patientIndexes_test[i+1]==patientIndexes_test[i]:
                next=predictions[i+1]
            else:
                next=0
            if i<len(predictions)-2:
                if patientIndexes_test[i+2]==patientIndexes_test[i]:
                    next1=predictions[i+2]
                else:
                    next1=0
            else:
                next1=0
        else:
            next=0
            next1=0
        predictions_1.append(np.mean([prev1,prev,current,next,next1]))
    predictions_1=np.array(predictions_1)
    return(predictions_1)

#function for evaluating the predicted classification labels
#outputs: lists of patient-wise accuracy, sensitivity, specificity, precision, and AUC values
def evaluatePreds(predictions,y_test,patientIndexes_test,trainPreds,y_train):

    fpr, tpr, thresholds = roc_curve(y_train, trainPreds, drop_intermediate=False)
    J_stats = tpr - fpr
    youden = thresholds[np.argmax(J_stats)]

    acc_list=[]
    sen_list=[]
    spe_list=[]
    pre_list=[]
    auc_list=[]
    
    for i in range(len(np.unique(patientIndexes_test))):
        subPredictions=np.array(predictions[patientIndexes_test==np.unique(patientIndexes_test)[i]],dtype=float)
        subLabels=np.array(y_test[patientIndexes_test==np.unique(patientIndexes_test)[i]],dtype=int)
        negLabels=-subLabels+1
        TP = np.sum(subLabels[subPredictions>=youden])
        FN = np.sum(subLabels[subPredictions<youden])
        FP = np.sum(negLabels[subPredictions>=youden])
        TN = np.sum(negLabels[subPredictions<youden])
        if TP+TN==0:
            acc=0
        else:
           acc = (TP+TN)/(TP+TN+FN+FP) 
        if TP==0:
            sen=0
            pre=0
        else:
            sen = TP/(TP+FN)
            pre = TP/(TP+FP)
        if TN==0:
            spe=0
        else:
            spe = TN/(TN+FP)
        acc_list.append(acc)
        spe_list.append(spe)
        if TP+FN!=0:
            sen_list.append(sen)
            pre_list.append(pre)
            fpr, tpr, thresholds = roc_curve(subLabels,subPredictions,drop_intermediate=False)
            auc1=auc(fpr, tpr)
            auc_list.append(auc1)
    return(acc_list,sen_list,spe_list,pre_list,auc_list)

#function for finding the indexes of the slices predicted as positive
def findPredictedSliceIndexes(predictions,trainPreds,y_train):

    fpr, tpr, thresholds = roc_curve(y_train, trainPreds, drop_intermediate=False)
    J_stats = tpr - fpr
    youden = thresholds[np.argmax(J_stats)]
    sliceIndexes=np.array(predictions>youden,dtype=int)
    return(sliceIndexes)

#function for computing patient-wise Dice scores
def computeDices(predictions,y_test,patientIndexes_test,threshold):

    dices2d=[]
    dices3d=[]
    TP=0
    FN=0
    FP=0
    TN=0
    for i in range(len(patientIndexes_test)):
        TP_new=np.sum(np.minimum(np.array(y_test[i]>=0.5,dtype=int),np.array(predictions[i][:,:,0]>=threshold,dtype=int)))
        FN_new=np.sum(np.minimum(np.array(y_test[i]>=0.5,dtype=int),np.array(predictions[i][:,:,0]<threshold,dtype=int)))
        FP_new=np.sum(np.minimum(np.array(y_test[i]<0.5,dtype=int),np.array(predictions[i][:,:,0]>=threshold,dtype=int)))
        TN_new=np.sum(np.minimum(np.array(y_test[i]<0.5,dtype=int),np.array(predictions[i][:,:,0]<threshold,dtype=int)))
        if 2*TP_new+FN_new+FP_new>0:
            dices2d.append(2*TP_new/(2*TP_new+FN_new+FP_new))
        else:
            dices2d.append(0)
        TP+=TP_new
        FN+=FN_new
        FP+=FP_new
        TN+=TN_new
        if i<len(patientIndexes_test)-1:
            if patientIndexes_test[i]!=patientIndexes_test[i+1]:
                if 2*TP+FN+FP>0:
                    dices3d.append(2*TP/(2*TP+FN+FP))
                else:
                    dices3d.append(0)
                TP=0
                FN=0
                FP=0
                TN=0
    dices3d.append(2*TP/(2*TP+FN+FP))
    return(dices2d,dices3d)

#functions for visualization
def get_all_edges(bool_img):
    """
    Get a list of all edges (where the value changes from True to False) in the 2D boolean image.
    The returned array edges has he dimension (n, 2, 2).
    Edge i connects the pixels edges[i, 0, :] and edges[i, 1, :].
    Note that the indices of a pixel also denote the coordinates of its lower left corner.
    """
    edges = []
    ii, jj = np.nonzero(bool_img)
    for i, j in zip(ii, jj):
        # North
        if j == bool_img.shape[1]-1 or not bool_img[i, j+1]:
            edges.append(np.array([[i, j+1],
                                   [i+1, j+1]]))
        # East
        if i == bool_img.shape[0]-1 or not bool_img[i+1, j]:
            edges.append(np.array([[i+1, j],
                                   [i+1, j+1]]))
        # South
        if j == 0 or not bool_img[i, j-1]:
            edges.append(np.array([[i, j],
                                   [i+1, j]]))
        # West
        if i == 0 or not bool_img[i-1, j]:
            edges.append(np.array([[i, j],
                                   [i, j+1]]))

    if not edges:
        return np.zeros((0, 2, 2))
    else:
        return np.array(edges)


def close_loop_edges(edges):
    """
    Combine the edges defined by 'get_all_edges' to closed loops around objects.
    If there are multiple disconnected objects a list of closed loops is returned.
    Note that it's expected that all the edges are part of exactly one loop (but not necessarily the same one).
    """

    loop_list = []
    while edges.size != 0:

        loop = [edges[0, 0], edges[0, 1]]  # Start with first edge
        edges = np.delete(edges, 0, axis=0)

        while edges.size != 0:
            # Get next edge (=edge with common node)
            ij = np.nonzero((edges == loop[-1]).all(axis=2))
            if ij[0].size > 0:
                i = ij[0][0]
                j = ij[1][0]
            else:
                loop.append(loop[0])
                # Uncomment to to make the start of the loop invisible when plotting
                # loop.append(loop[1])
                break

            loop.append(edges[i, (j + 1) % 2, :])
            edges = np.delete(edges, i, axis=0)

        loop_list.append(np.array(loop))

    return loop_list


def plot_outlines(bool_img, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    edges = get_all_edges(bool_img=bool_img)
    edges = edges - 0.5  # convert indices to coordinates; TODO adjust according to image extent
    outlines = close_loop_edges(edges=edges)
    cl = LineCollection(outlines, **kwargs)
    ax.add_collection(cl)

In [ ]:
#collect the number of slices and the pixel spacings
folder_path='D:/img/rpe/RPE-SEGMENTIT'
files=os.listdir(folder_path)
img_height=128
coronalSlices=[]
transaxialSlices=[]
pixelSpacings=[]
transPixelSpacings=[]
for i in range(len(files)):
    pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
    img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
    coronalSlices.append(img3d.shape[0])
    transaxialSlices.append(img3d.shape[2])
    pixelSpacings.append(voxelSize[0])
    transPixelSpacings.append(voxelSize[2])

In [ ]:
#print information about the lists collected above
y=transPixelSpacings
print(np.min(y))
print(np.max(y))
print(np.mean(y))
print(np.std(y))

In [ ]:
#collect the 2D image slices, their ground-truth labels, and the patient indexes
direction=['coronal','sagittal','transaxial'][0]
folder_path='D:/img/rpe/RPE-SEGMENTIT'
files=os.listdir(folder_path)
img_height=128
slices=[]
labels=[]
patientIndexes=[]
for i in range(len(files)):
    mask_path='{}/{}'.format(folder_path,files[i])
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
    img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
    if direction=='coronal':
        for j in range(mask.shape[1]):
            slices.append(preprocessImgSlice(img3d[:,j,:],img_height))
            labels.append(int(np.max(mask[:,j,:])>0.5))
            patientIndexes.append(int(files[i][0:4]))
    if direction=='sagittal':
        for j in range(mask.shape[0]):
            slices.append(preprocessImgSlice(img3d[j,:,:],img_height))
            labels.append(int(np.max(mask[j,:,:])>0.5))
            patientIndexes.append(int(files[i][0:4]))
    if direction=='transaxial':
        for j in range(mask.shape[2]):
            slices.append(preprocessImgSlice(img3d[:,:,j],img_height))
            labels.append(int(np.max(mask[:,:,j])>0.5))
            patientIndexes.append(int(files[i][0:4]))
print(len(np.unique(patientIndexes)))
print(len(labels))
print(np.sum(labels))
#244 patients
#coronal: 100,400 slices, 9,321 positive slices
#sagittal: 100,400 slices, 17,931 positive slices
#transaxial: 11,197 slices, 3,378 positive slices

In [ ]:
#train encoderCNNs and save the results
for k in range(5):
    x_train,y_train,x_test,y_test,patientIndexes_train,patientIndexes_test=fiveFoldCrossValidation(slices,labels,patientIndexes,k)
    numberOfEpochs=10
    startTime=perf_counter()
    predictions,trainPreds=predict(x_train,y_train,x_test,numberOfEpochs)
    endTime=perf_counter()
    processingTime=endTime-startTime
    np.savetxt('rcls_{}_{}_time.txt'.format(direction,k),np.array([processingTime]))
    np.savetxt('rcls_{}_{}_predictions.txt'.format(direction,k),predictions)
    np.savetxt('rcls_{}_{}_trainPreds.txt'.format(direction,k),trainPreds)
    np.savetxt('rcls_{}_{}_patientIndexes_test.txt'.format(direction,k),patientIndexes_test)
    np.savetxt('rcls_{}_{}_patientIndexes_train.txt'.format(direction,k),patientIndexes_train)
    np.savetxt('rcls_{}_{}_y_test.txt'.format(direction,k),y_test)
    np.savetxt('rcls_{}_{}_y_train.txt'.format(direction,k),y_train)

In [ ]:
#compute the evaluation metrics
for direction in ['coronal','sagittal','transaxial']:
    acc_list_1=[]
    sen_list_1=[]
    spe_list_1=[]
    pre_list_1=[]
    auc_list_1=[]
    for k in range(5):
        processingTimes=np.loadtxt('rcls_{}_{}_time.txt'.format(direction,k))
        predictions=np.loadtxt('rcls_{}_{}_predictions.txt'.format(direction,k))
        y_test=np.loadtxt('rcls_{}_{}_y_test.txt'.format(direction,k))
        patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format(direction,k))
        trainPreds=np.loadtxt('rcls_{}_{}_trainPreds.txt'.format(direction,k))
        y_train=np.loadtxt('rcls_{}_{}_y_train.txt'.format(direction,k))
        patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format(direction,k))
        predictions1=postProcess_new(predictions,patientIndexes_test)
        trainPreds1=postProcess_new(trainPreds,patientIndexes_train)
        acc_list,sen_list,spe_list,pre_list,auc_list=evaluatePreds(predictions1,y_test,patientIndexes_test,trainPreds1,y_train)
        acc_list_1=acc_list_1+acc_list
        sen_list_1=sen_list_1+sen_list
        spe_list_1=spe_list_1+spe_list
        pre_list_1=pre_list_1+pre_list
        auc_list_1=auc_list_1+auc_list
    print(round(np.mean(acc_list_1),3),round(np.std(acc_list_1),3))
    print(round(np.mean(sen_list_1),3),round(np.std(sen_list_1),3))
    print(round(np.mean(spe_list_1),3),round(np.std(spe_list_1),3))
    print(round(np.mean(pre_list_1),3),round(np.std(pre_list_1),3))
    print(round(np.mean(auc_list_1),3),round(np.std(auc_list_1),3))
    if direction=='coronal':
        auc_list_coronal=auc_list_1
    if direction=='sagittal':
        auc_list_sagittal=auc_list_1
    if direction=='transaxial':
        auc_list_transaxial=auc_list_1

#test if there are significant differences in the AUC
print(wilcoxon(auc_list_coronal,auc_list_sagittal))
print(wilcoxon(auc_list_transaxial,auc_list_coronal))
print(wilcoxon(auc_list_transaxial,auc_list_sagittal))

In [ ]:
#compute the numbers of voxels in binary masks, real RPE volumes, and their percentage of the mask
folder_path='D:/img/rpe/RPE-SEGMENTIT'
files=os.listdir(folder_path)
voxelsInMask=[]
volumes_patient=[]
percentageOfRPE=[]
structure=np.ones((3,3,3),dtype=int)
for i in range(len(files)):
    mask_path='{}/{}'.format(folder_path,files[i])
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
    img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)   
    labeled, ncomponents = label(np.array(mask>0.5,dtype=int),structure)
    #components.append(ncomponents)
    #for u in range(1,ncomponents+1):
    #    img0=np.array(labeled==u,dtype=int)
    #    voxelsInComponent.append(np.sum(img0))
    #    volumes_component.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(img0))
    volumes_patient.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(np.array(mask>0.5,dtype=int)))
    voxelsInMask.append(np.sum(np.array(mask>0.5,dtype=int)))
    percentageOfRPE.append(np.sum(np.array(mask>0.5,dtype=int))/mask.shape[0]/mask.shape[1]/mask.shape[2])
    print(i)
real_volumes_patient=volumes_patient

In [ ]:
y=np.array(percentageOfRPE)*100
print(np.mean(y))
print(np.std(y))
print(np.min(y))
print(np.median(y))
print(np.max(y))

In [ ]:
#save indexes of the slices predicted as positive
for direction in ['coronal','sagittal','transaxial']:
    for k in range(5):
        processingTimes=np.loadtxt('rcls_{}_{}_time.txt'.format(direction,k))
        predictions=np.loadtxt('rcls_{}_{}_predictions.txt'.format(direction,k))
        y_test=np.loadtxt('rcls_{}_{}_y_test.txt'.format(direction,k))
        patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format(direction,k))
        trainPreds=np.loadtxt('rcls_{}_{}_trainPreds.txt'.format(direction,k))
        y_train=np.loadtxt('rcls_{}_{}_y_train.txt'.format(direction,k))
        patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format(direction,k))
        predictions1=postProcess_new(predictions,patientIndexes_test)
        trainPreds1=postProcess_new(trainPreds,patientIndexes_train)
        sliceIndexes=findPredictedSliceIndexes(predictions1,trainPreds1,y_train)
        np.savetxt('rcls_{}_{}_sliceIndexes.txt'.format(direction,k),sliceIndexes)
for direction in ['coronal','sagittal','transaxial']:
    for k in range(5):
        processingTimes=np.loadtxt('rcls_{}_{}_time.txt'.format(direction,k))
        predictions=np.loadtxt('rcls_{}_{}_predictions.txt'.format(direction,k))
        y_test=np.loadtxt('rcls_{}_{}_y_test.txt'.format(direction,k))
        patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format(direction,k))
        trainPreds=np.loadtxt('rcls_{}_{}_trainPreds.txt'.format(direction,k))
        y_train=np.loadtxt('rcls_{}_{}_y_train.txt'.format(direction,k))
        patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format(direction,k))
        predictions1=postProcess_new(predictions,patientIndexes_test)
        trainPreds1=postProcess_new(trainPreds,patientIndexes_train)
        sliceIndexes=findPredictedSliceIndexes(trainPreds1,trainPreds1,y_train)
        np.savetxt('rcls_{}_{}_trainSliceIndexes.txt'.format(direction,k),sliceIndexes)

In [ ]:
#compute the Dices between the predicted 3D bounding boxes and the real volumes
folder_path='D:/img/rpe/RPE-SEGMENTIT'
files=os.listdir(folder_path)
dices3d=[]
components=[]
voxelsInComponent=[]
volumes_component=[]
volumes_patient=[]
volumeContained=[]
dicesForBoxes=[]
structure=np.ones((3,3,3),dtype=int)
for i in range(len(files)):
    mask_path='{}/{}'.format(folder_path,files[i])
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
    img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
    predictedMask=np.zeros(img3d.shape)
    index=int(files[i][0:4])
    k=index % 5
    direction='coronal'
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format(direction,k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format(direction,k))
    for j in range(mask.shape[1]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[:,j,:]+=1
    direction='sagittal'
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format(direction,k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format(direction,k))
    for j in range(mask.shape[0]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[j,:,:]+=1
    direction='transaxial'
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format(direction,k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format(direction,k))
    for j in range(mask.shape[2]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[:,:,j]+=1
    predictedMask=np.array(predictedMask>2.5,dtype=int)
    TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(predictedMask>=0.5,dtype=int)))
    FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(predictedMask<0.5,dtype=int)))
    FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(predictedMask>=0.5,dtype=int)))
    TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(predictedMask<0.5,dtype=int)))
    dice=2*TP/(2*TP+FN+FP)
    dices3d.append(dice)
    volumeContained.append(TP/(TP+FN))
    labeled, ncomponents = label(predictedMask,structure)
    components.append(ncomponents)
    for u in range(1,ncomponents+1):
        img0=np.array(labeled==u,dtype=int)
        voxelsInComponent.append(np.sum(img0))
        volumes_component.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(img0))
    volumes_patient.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(predictedMask))
    groundTruthBoxes=np.zeros(img3d.shape)
    for j in range(mask.shape[0]):
        groundTruthBoxes[j,:,:]+=np.max(mask[j,:,:])
    for j in range(mask.shape[1]):
        groundTruthBoxes[:,j,:]+=np.max(mask[:,j,:])
    for j in range(mask.shape[2]):
        groundTruthBoxes[:,:,j]+=np.max(mask[:,:,j])
    groundTruthBoxes=np.array(groundTruthBoxes>2.5,dtype=int)
    TP=np.sum(np.minimum(np.array(groundTruthBoxes>=0.5,dtype=int),np.array(predictedMask>=0.5,dtype=int)))
    FN=np.sum(np.minimum(np.array(groundTruthBoxes>=0.5,dtype=int),np.array(predictedMask<0.5,dtype=int)))
    FP=np.sum(np.minimum(np.array(groundTruthBoxes<0.5,dtype=int),np.array(predictedMask>=0.5,dtype=int)))
    TN=np.sum(np.minimum(np.array(groundTruthBoxes<0.5,dtype=int),np.array(predictedMask<0.5,dtype=int)))
    dicesForBoxes.append(2*TP/(2*TP+FN+FP))
    print(i)

In [ ]:
y=np.array(dices3d)
print(np.mean(y))
print(np.std(y))
print(np.min(y))
print(np.median(y))
print(np.max(y))

In [ ]:
#plot results
plt.imshow(np.mean(img3d,axis=0),cmap='gray')
plt.show()
plt.imshow(np.mean(mask,axis=0),cmap='gray')
plt.show()
plt.imshow(np.mean(predictedMask,axis=0),cmap='gray')
plt.show()
plt.imshow(np.mean(img3d,axis=1),cmap='gray')
plt.show()
plt.imshow(np.mean(mask,axis=1),cmap='gray')
plt.show()
plt.imshow(np.mean(predictedMask,axis=1),cmap='gray')
plt.show()
plt.imshow(np.mean(img3d,axis=2),cmap='gray')
plt.show()
plt.imshow(np.mean(mask,axis=2),cmap='gray')
plt.show()
plt.imshow(np.mean(predictedMask,axis=2),cmap='gray')
plt.show()

In [ ]:
#perform the segmentation with U-Nets and save the trained models
for direction in ['coronal','sagittal','transaxial']:

    for k in range(5):

        folder_path='D:/img/rpe/RPE-SEGMENTIT'
        files=os.listdir(folder_path)
        slices=[]
        masks=[]
        patientIndexes=[]
        for i in range(len(files)):
            mask_path='{}/{}'.format(folder_path,files[i])
            mask=nib.load(mask_path)
            mask=mask.get_fdata()
            pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
            img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
            predictedMask=np.zeros(img3d.shape)
            index=int(files[i][0:4])
            if index % 5 !=k:
                trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('coronal',k))
                patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('coronal',k))
                for j in range(mask.shape[1]):
                    if trainSliceIndexes[patientIndexes_train==index][j]==1:
                        predictedMask[:,j,:]+=1
                trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('sagittal',k))
                patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('sagittal',k))
                for j in range(mask.shape[0]):
                    if trainSliceIndexes[patientIndexes_train==index][j]==1:
                        predictedMask[j,:,:]+=1
                trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('transaxial',k))
                patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('transaxial',k))
                for j in range(mask.shape[2]):
                    if trainSliceIndexes[patientIndexes_train==index][j]==1:
                        predictedMask[:,:,j]+=1
            predictedMask=np.array(predictedMask>2.5,dtype=int)
            if np.max(predictedMask)>0:
                if direction=='coronal':
                    for j in range(mask.shape[1]):
                        if np.max(predictedMask[:,j,:])>0:
                            for i1 in range(0,mask.shape[0]-64,32):
                                if np.sum(predictedMask[i1:(i1+64),j,:])>min(1000,np.sum(predictedMask[:,j,:])/3):
                                    slices.append(preprocessImgSlice(img3d[i1:(i1+64),j,:],64))
                                    masks.append(np.array(cv2.resize(mask[i1:(i1+64),j,:],(64,64))>0.5,dtype=int))
                                    patientIndexes.append(int(files[i][0:4]))
                if direction=='sagittal':
                    for j in range(mask.shape[0]):
                        if np.max(predictedMask[j,:,:])>0:
                            for i1 in range(0,mask.shape[1]-64,32):
                                if np.sum(predictedMask[j,i1:(i1+64),:])>min(1000,np.sum(predictedMask[j,:,:])/3):
                                    slices.append(preprocessImgSlice(img3d[j,i1:(i1+64),:],64))
                                    masks.append(np.array(cv2.resize(mask[j,i1:(i1+64),:],(64,64))>0.5,dtype=int))
                                    patientIndexes.append(int(files[i][0:4]))
                if direction=='transaxial':            
                    for j in range(mask.shape[2]):
                        if np.max(predictedMask[:,:,j])>0:
                            for i1 in range(0,mask.shape[0]-64,32):
                                for j1 in range(0,mask.shape[1]-64,32):
                                    if np.sum(predictedMask[i1:(i1+64),j1:(j1+64),j])>min(1000,np.sum(predictedMask[:,:,j])/3):
                                        slices.append(preprocessImgSlice(img3d[i1:(i1+64),j1:(j1+64),j],64))
                                        masks.append(mask[i1:(i1+64),j1:(j1+64),j])
                                        patientIndexes.append(int(files[i][0:4]))
            print(i)
        print(k,direction,len(masks))

        x_train,y_train,x_test,y_test,patientIndexes_train,patientIndexes_test=fiveFoldCrossValidation(slices,masks,patientIndexes,k)
        print(len(x_train))
        print(len(x_test))

        #the segmentation model (a standard U-Net model with some layers removed so that it suits for 64*64 data)
        inputs = tf.keras.layers.Input(shape=(64,64,1))

        #Contraction path
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
        c1 = tf.keras.layers.Dropout(0.1)(c1)
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
        p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

        c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
        c2 = tf.keras.layers.Dropout(0.1)(c2)
        c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
        p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

        c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
        c3 = tf.keras.layers.Dropout(0.2)(c3)
        c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)

        #Expansive path
        u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c3)
        u8 = tf.keras.layers.concatenate([u8, c2])
        c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
        c8 = tf.keras.layers.Dropout(0.1)(c8)
        c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

        u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
        u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
        c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
        c9 = tf.keras.layers.Dropout(0.1)(c9)
        c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

        outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)#or linear

        segModel = tf.keras.Model(inputs=inputs, outputs=outputs)

        numberOfEpochs=30
        startTime=perf_counter()
        segModel.compile(optimizer=tf.keras.optimizers.Adam(1e-3),loss=tf.keras.losses.Dice())#or tf.keras.losses.BinaryCrossentropy()
        history=segModel.fit(x=x_train,y=y_train,epochs=numberOfEpochs,shuffle=True)
        plt.plot(history.history['loss'],color='blue')
        endTime=perf_counter()
        processingTime=endTime-startTime
        np.savetxt('rcls_seg_{}_{}_time.txt'.format(direction,k),np.array([processingTime]))
        segModel.save('segModel_{}_{}.keras'.format(direction,k),include_optimizer=True)

In [ ]:
#compute the resulting Dice scores
segModel1_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',0))
segModel1_cor.compile(optimizer='adam',loss='Dice')
segModel2_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',1))
segModel2_cor.compile(optimizer='adam',loss='Dice')
segModel3_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',2))
segModel3_cor.compile(optimizer='adam',loss='Dice')
segModel4_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',3))
segModel4_cor.compile(optimizer='adam',loss='Dice')
segModel5_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',4))
segModel5_cor.compile(optimizer='adam',loss='Dice')

segModel1_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',0))
segModel1_sag.compile(optimizer='adam',loss='Dice')
segModel2_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',1))
segModel2_sag.compile(optimizer='adam',loss='Dice')
segModel3_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',2))
segModel3_sag.compile(optimizer='adam',loss='Dice')
segModel4_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',3))
segModel4_sag.compile(optimizer='adam',loss='Dice')
segModel5_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',4))
segModel5_sag.compile(optimizer='adam',loss='Dice')

segModel1_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',0))
segModel1_trans.compile(optimizer='adam',loss='Dice')
segModel2_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',1))
segModel2_trans.compile(optimizer='adam',loss='Dice')
segModel3_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',2))
segModel3_trans.compile(optimizer='adam',loss='Dice')
segModel4_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',3))
segModel4_trans.compile(optimizer='adam',loss='Dice')
segModel5_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',4))
segModel5_trans.compile(optimizer='adam',loss='Dice')

direction=['coronal','sagittal','transaxial'][2]
folder_path='D:/img/rpe/RPE-SEGMENTIT'
files=os.listdir(folder_path)
dices_cor=[]
dices_sag=[]
dices_trans=[]
dices_2_5d=[]
pred_volumes_patient=[]
trans_volumes_patient=[]
real_volumes_patient=[]
for i in range(len(files)):
    mask_path='{}/{}'.format(folder_path,files[i])
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
    img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
    predictedMask=np.zeros(img3d.shape)
    index=int(files[i][0:4])
    k=index % 5
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('coronal',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('coronal',k))
    for j in range(mask.shape[1]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[:,j,:]+=1
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('sagittal',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('sagittal',k))
    for j in range(mask.shape[0]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[j,:,:]+=1
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('transaxial',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('transaxial',k))
    for j in range(mask.shape[2]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[:,:,j]+=1
    predictedMask=np.array(predictedMask>2.5,dtype=int)
    newMask_2_5d=np.zeros(predictedMask.shape)
    for d in ['coronal','sagittal','transaxial']:
        newMask=np.zeros(predictedMask.shape)
        denominators=np.zeros(predictedMask.shape)
        if np.max(predictedMask)>0:
            if d=='coronal':
                for j in range(mask.shape[1]):
                    if np.max(predictedMask[:,j,:])>0:
                        for i1 in range(0,mask.shape[0]-64,32):
                            if np.sum(predictedMask[i1:(i1+64),j,:])>min(1000,np.sum(predictedMask[:,j,:])/3):
                                slice=preprocessImgSlice(img3d[i1:(i1+64),j,:],64)
                                if k==0:
                                    pred=segModel1_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==1:
                                    pred=segModel2_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==2:
                                    pred=segModel3_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==3:
                                    pred=segModel4_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==4:
                                    pred=segModel5_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                pred=cv2.resize(pred,(mask.shape[2],64))
                                newMask[i1:(i1+64),j,:]+=pred
                                denominators[i1:(i1+64),j,:]+=1
            if d=='sagittal':
                for j in range(mask.shape[0]):
                    if np.max(predictedMask[j,:,:])>0:
                        for i1 in range(0,mask.shape[1]-64,32):
                            if np.sum(predictedMask[j,i1:(i1+64),:])>min(1000,np.sum(predictedMask[:,j,:])/3):
                                slice=preprocessImgSlice(img3d[j,i1:(i1+64),:],64)
                                if k==0:
                                    pred=segModel1_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==1:
                                    pred=segModel2_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==2:
                                    pred=segModel3_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==3:
                                    pred=segModel4_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==4:
                                    pred=segModel5_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                pred=cv2.resize(pred,(mask.shape[2],64))
                                newMask[j,i1:(i1+64),:]+=pred
                                denominators[j,i1:(i1+64),:]+=1
            if d=='transaxial':
                for j in range(mask.shape[2]):
                    if np.max(predictedMask[:,:,j])>0:
                        for i1 in range(0,mask.shape[0]-64,32):
                            for j1 in range(0,mask.shape[0]-64,32):
                                if np.sum(predictedMask[i1:(i1+64),j1:(j1+64),j])>min(1000,np.sum(predictedMask[:,:,j])/3):
                                    slice=preprocessImgSlice(img3d[i1:(i1+64),j1:(j1+64),j],64)
                                    if k==0:
                                        pred=segModel1_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    if k==1:
                                        pred=segModel2_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    if k==2:
                                        pred=segModel3_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    if k==3:
                                        pred=segModel4_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    if k==4:
                                        pred=segModel5_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    newMask[i1:(i1+64),j1:(j1+64),j]+=pred
                                    denominators[i1:(i1+64),j1:(j1+64),j]+=1
        denominators[denominators==0]=1
        newMask=newMask/denominators
        newMask=np.array(newMask>0.5,dtype=int)
        TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
        FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
        FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
        TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
        dice=2*TP/(2*TP+FN+FP)
        if d=='coronal':
            dices_cor.append(dice)
        if d=='sagittal':
            dices_sag.append(dice)
        if d=='transaxial':
            dices_trans.append(dice)
            trans_volumes_patient.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(newMask))
        newMask_2_5d+=newMask
    newMask=np.array(newMask_2_5d>1.5,dtype=int)
    TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
    FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
    FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
    TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
    dice=2*TP/(2*TP+FN+FP)
    dices_2_5d.append(dice)
    print(i)
    pred_volumes_patient.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(newMask))
    real_volumes_patient.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(mask))
np.savetxt('rcls_dices_cor.txt',dices_cor)
np.savetxt('rcls_dices_sag.txt',dices_sag)
np.savetxt('rcls_dices_trans.txt',dices_trans)
np.savetxt('rcls_dices_2_5d.txt',dices_2_5d)                                                    
np.savetxt('rcls_volumes_trans.txt',trans_volumes_patient)
np.savetxt('rcls_volumes_2_5d.txt',pred_volumes_patient)
np.savetxt('rcls_volumes_real.txt',real_volumes_patient)

In [ ]:
#load results
dices_cor=np.loadtxt('rcls_dices_cor.txt')
dices_sag=np.loadtxt('rcls_dices_sag.txt')
dices_trans=np.loadtxt('rcls_dices_trans.txt')
dices_2_5d=np.loadtxt('rcls_dices_2_5d.txt')
real_volumes_patient=np.loadtxt('rcls_volumes_real.txt')

In [ ]:
y=dices_2_5d
print(round(np.mean(y),3),round(np.std(y),3))
print(round(np.min(y),3))
print(np.sum(y==0))
print(round(np.median(y),3))
print(round(np.max(y),3))

In [ ]:
print(pearsonr(dices_trans,real_volumes_patient))

In [ ]:
print(np.min(real_volumes_patient[dices_trans==0]))

In [ ]:
np.sum(real_volumes_patient>3000)

In [ ]:
print(np.mean(dices_trans[real_volumes_patient>3000]))
print(np.std(dices_trans[real_volumes_patient>3000]))
print(np.median(dices_trans[real_volumes_patient>3000]))

In [ ]:
plt.scatter(real_volumes_patient,dices_trans,color='blue')

In [ ]:
direction='sagittal'
times=[]
for k in range(5):
    #times.append(np.loadtxt('rcls_{}_{}_time.txt'.format(direction,k))/60)
    #np.loadtxt('rcls_seg_{}_{}_time.txt'.format(direction,k))
    time=np.loadtxt('rcls_seg_{}_{}_time.txt'.format('coronal',k))+np.loadtxt('rcls_seg_{}_{}_time.txt'.format('sagittal',k))+np.loadtxt('rcls_seg_{}_{}_time.txt'.format('transaxial',k))
    times.append(time/60)
print(round(np.mean(times),2),round(np.std(times),2))

In [ ]:
print(wilcoxon(dices_trans,dices_cor))
print(wilcoxon(dices_trans,dices_sag))
print(wilcoxon(dices_trans,dices_2_5d))

In [ ]:
print(round(np.mean(dices_cor),3),round(np.std(dices_cor),3))
print(round(np.mean(dices_sag),3),round(np.std(dices_sag),3))
print(round(np.mean(dices_trans),3),round(np.std(dices_trans),3))
print(round(np.mean(dices_2_5d),3),round(np.std(dices_2_5d),3))

In [ ]:
#try segmentation without bounding boxes (this did not converge)
for direction in ['transaxial','coronal','sagittal']:

    for k in range(5):

        folder_path='D:/img/rpe/RPE-SEGMENTIT'
        files=os.listdir(folder_path)
        slices=[]
        masks=[]
        patientIndexes=[]
        for i in range(len(files)):
            mask_path='{}/{}'.format(folder_path,files[i])
            mask=nib.load(mask_path)
            mask=mask.get_fdata()
            pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
            img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
            index=int(files[i][0:4])
            if index % 5 !=k:
                if direction=='coronal':
                    trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('coronal',k))
                    patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('coronal',k))
                    for j in range(mask.shape[1]):
                        if trainSliceIndexes[patientIndexes_train==index][j]==1:
                            slices.append(preprocessImgSlice(img3d[:,j,:],64))
                            masks.append(np.array(cv2.resize(mask[:,j,:],(64,64))>0.5,dtype=int))
                            patientIndexes.append(int(files[i][0:4]))
                if direction=='sagittal':
                    trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('sagittal',k))
                    patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('sagittal',k))
                    for j in range(mask.shape[0]):
                        if trainSliceIndexes[patientIndexes_train==index][j]==1:
                            slices.append(preprocessImgSlice(img3d[j,:,:],64))
                            masks.append(np.array(cv2.resize(mask[j,:,:],(64,64))>0.5,dtype=int))
                            patientIndexes.append(int(files[i][0:4]))
                if direction=='transaxial':
                    trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('transaxial',k))
                    patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('transaxial',k))
                    for j in range(mask.shape[2]):
                        if trainSliceIndexes[patientIndexes_train==index][j]==1:
                            slices.append(preprocessImgSlice(img3d[:,:,j],64))
                            masks.append(np.array(cv2.resize(mask[:,:,j],(64,64))>0.5,dtype=int))
                            patientIndexes.append(int(files[i][0:4]))
            print(i)
        print(k,direction,len(masks))

        x_train,y_train,x_test,y_test,patientIndexes_train,patientIndexes_test=fiveFoldCrossValidation(slices,masks,patientIndexes,k)
        print(len(x_train))
        print(len(x_test))

        #the segmentation model (a standard U-Net model with some layers removed so that it suits for 64*64 data)
        inputs = tf.keras.layers.Input(shape=(64,64,1))

        #Contraction path
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
        c1 = tf.keras.layers.Dropout(0.1)(c1)
        c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
        p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

        c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
        c2 = tf.keras.layers.Dropout(0.1)(c2)
        c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
        p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

        c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
        c3 = tf.keras.layers.Dropout(0.2)(c3)
        c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)

        #Expansive path
        u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c3)
        u8 = tf.keras.layers.concatenate([u8, c2])
        c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
        c8 = tf.keras.layers.Dropout(0.1)(c8)
        c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

        u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
        u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
        c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
        c9 = tf.keras.layers.Dropout(0.1)(c9)
        c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

        outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)#or linear

        segModel = tf.keras.Model(inputs=inputs, outputs=outputs)

        numberOfEpochs=30
        startTime=perf_counter()
        segModel.compile(optimizer=tf.keras.optimizers.Adam(1e-3),loss=tf.keras.losses.Dice())#or tf.keras.losses.BinaryCrossentropy()
        history=segModel.fit(x=x_train,y=y_train,epochs=numberOfEpochs,shuffle=True)
        plt.plot(history.history['loss'],color='blue')
        endTime=perf_counter()
        processingTime=endTime-startTime
        np.savetxt('rcls_control_seg_{}_{}_time.txt'.format(direction,k),np.array([processingTime]))
        segModel.save('segModel_control_{}_{}.keras'.format(direction,k),include_optimizer=True)

In [ ]:
print(0.001*np.mean(abs(np.array(trans_volumes_patient)-np.array(real_volumes_patient))))
print(np.mean((0.001*np.array(trans_volumes_patient)-0.001*np.array(real_volumes_patient))**2)**0.5)
print(np.mean(abs(np.array(trans_volumes_patient)-np.array(real_volumes_patient))/np.array(real_volumes_patient)))

In [ ]:
plt.scatter(0.001*np.array(real_volumes_patient),0.001*np.array(trans_volumes_patient),color='blue')
plt.xlabel('Real RPE volume (mL)')
plt.ylabel('Predicted RPE volume (mL)')

In [ ]:
print(pearsonr(trans_volumes_patient,real_volumes_patient))

In [ ]:
#compute the number of slices
slices_cor=[]
slices_sag=[]
slices_trans=[]
k_indexes=[]

folder_path='D:/img/rpe/RPE-SEGMENTIT'
files=os.listdir(folder_path)
for i in range(len(files)):
    mask_path='{}/{}'.format(folder_path,files[i])
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    slices_cor.append(mask.shape[1])
    slices_sag.append(mask.shape[0])
    slices_trans.append(mask.shape[2])

In [ ]:
#compute the training set sizes
for direction in ['coronal','sagittal','transaxial']:

    for k in range(5):

        folder_path='D:/img/rpe/RPE-SEGMENTIT'
        files=os.listdir(folder_path)
        slices=[]
        masks=[]
        patientIndexes=[]
        for i in range(len(files)):
            mask_path='{}/{}'.format(folder_path,files[i])
            mask=nib.load(mask_path)
            mask=mask.get_fdata()
            pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
            img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
            predictedMask=np.zeros(img3d.shape)
            index=int(files[i][0:4])
            if index % 5 !=k:
                trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('coronal',k))
                patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('coronal',k))
                for j in range(mask.shape[1]):
                    if trainSliceIndexes[patientIndexes_train==index][j]==1:
                        predictedMask[:,j,:]+=1
                trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('sagittal',k))
                patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('sagittal',k))
                for j in range(mask.shape[0]):
                    if trainSliceIndexes[patientIndexes_train==index][j]==1:
                        predictedMask[j,:,:]+=1
                trainSliceIndexes=np.loadtxt('rcls_{}_{}_trainSliceIndexes.txt'.format('transaxial',k))
                patientIndexes_train=np.loadtxt('rcls_{}_{}_patientIndexes_train.txt'.format('transaxial',k))
                for j in range(mask.shape[2]):
                    if trainSliceIndexes[patientIndexes_train==index][j]==1:
                        predictedMask[:,:,j]+=1
            predictedMask=np.array(predictedMask>2.5,dtype=int)
            if np.max(predictedMask)>0:
                if direction=='coronal':
                    for j in range(mask.shape[1]):
                        if np.max(predictedMask[:,j,:])>0:
                            for i1 in range(0,mask.shape[0]-64,32):
                                if np.sum(predictedMask[i1:(i1+64),j,:])>min(1000,np.sum(predictedMask[:,j,:])/3):
                                    slices.append(preprocessImgSlice(img3d[i1:(i1+64),j,:],64))
                                    masks.append(np.array(cv2.resize(mask[i1:(i1+64),j,:],(64,64))>0.5,dtype=int))
                                    patientIndexes.append(int(files[i][0:4]))
                if direction=='sagittal':
                    for j in range(mask.shape[0]):
                        if np.max(predictedMask[j,:,:])>0:
                            for i1 in range(0,mask.shape[1]-64,32):
                                if np.sum(predictedMask[j,i1:(i1+64),:])>min(1000,np.sum(predictedMask[j,:,:])/3):
                                    slices.append(preprocessImgSlice(img3d[j,i1:(i1+64),:],64))
                                    masks.append(np.array(cv2.resize(mask[j,i1:(i1+64),:],(64,64))>0.5,dtype=int))
                                    patientIndexes.append(int(files[i][0:4]))
                if direction=='transaxial':            
                    for j in range(mask.shape[2]):
                        if np.max(predictedMask[:,:,j])>0:
                            for i1 in range(0,mask.shape[0]-64,32):
                                for j1 in range(0,mask.shape[1]-64,32):
                                    if np.sum(predictedMask[i1:(i1+64),j1:(j1+64),j])>min(1000,np.sum(predictedMask[:,:,j])/3):
                                        slices.append(preprocessImgSlice(img3d[i1:(i1+64),j1:(j1+64),j],64))
                                        masks.append(mask[i1:(i1+64),j1:(j1+64),j])
                                        patientIndexes.append(int(files[i][0:4]))
        print(k,direction,len(masks))

In [ ]:
#compute the numbers of 64*64 squares used as inputs for the segmentation
squares_cor=[]
squares_sag=[]
squares_trans=[]

for i in range(len(files)):
    mask_path='{}/{}'.format(folder_path,files[i])
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
    img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
    predictedMask=np.zeros(img3d.shape)
    index=int(files[i][0:4])
    k=index % 5
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('coronal',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('coronal',k))
    for j in range(mask.shape[1]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[:,j,:]+=1
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('sagittal',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('sagittal',k))
    for j in range(mask.shape[0]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[j,:,:]+=1
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('transaxial',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('transaxial',k))
    for j in range(mask.shape[2]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[:,:,j]+=1
    predictedMask=np.array(predictedMask>2.5,dtype=int)
    newMask_2_5d=np.zeros(predictedMask.shape)
    cor_squ=0
    sag_squ=0
    trans_squ=0
    for d in ['coronal','sagittal','transaxial']:
        if np.max(predictedMask)>0:
            if d=='coronal':
                for j in range(mask.shape[1]):
                    if np.max(predictedMask[:,j,:])>0:
                        for i1 in range(0,mask.shape[0]-64,32):
                            if np.sum(predictedMask[i1:(i1+64),j,:])>min(1000,np.sum(predictedMask[:,j,:])/3):
                                cor_squ+=1
            if d=='sagittal':
                for j in range(mask.shape[0]):
                    if np.max(predictedMask[j,:,:])>0:
                        for i1 in range(0,mask.shape[1]-64,32):
                            if np.sum(predictedMask[j,i1:(i1+64),:])>min(1000,np.sum(predictedMask[:,j,:])/3):
                                sag_squ+=1
            if d=='transaxial':
                for j in range(mask.shape[2]):
                    if np.max(predictedMask[:,:,j])>0:
                        for i1 in range(0,mask.shape[0]-64,32):
                            for j1 in range(0,mask.shape[0]-64,32):
                                if np.sum(predictedMask[i1:(i1+64),j1:(j1+64),j])>min(1000,np.sum(predictedMask[:,:,j])/3):
                                    trans_squ+=1
    squares_cor.append(cor_squ)
    squares_sag.append(sag_squ)
    squares_trans.append(trans_squ)

In [ ]:
#create the predicted segmentation masks for a specific patient
segModel1_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',0))
segModel1_cor.compile(optimizer='adam',loss='Dice')
segModel2_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',1))
segModel2_cor.compile(optimizer='adam',loss='Dice')
segModel3_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',2))
segModel3_cor.compile(optimizer='adam',loss='Dice')
segModel4_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',3))
segModel4_cor.compile(optimizer='adam',loss='Dice')
segModel5_cor=keras.models.load_model('segModel_{}_{}.keras'.format('coronal',4))
segModel5_cor.compile(optimizer='adam',loss='Dice')

segModel1_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',0))
segModel1_sag.compile(optimizer='adam',loss='Dice')
segModel2_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',1))
segModel2_sag.compile(optimizer='adam',loss='Dice')
segModel3_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',2))
segModel3_sag.compile(optimizer='adam',loss='Dice')
segModel4_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',3))
segModel4_sag.compile(optimizer='adam',loss='Dice')
segModel5_sag=keras.models.load_model('segModel_{}_{}.keras'.format('sagittal',4))
segModel5_sag.compile(optimizer='adam',loss='Dice')

segModel1_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',0))
segModel1_trans.compile(optimizer='adam',loss='Dice')
segModel2_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',1))
segModel2_trans.compile(optimizer='adam',loss='Dice')
segModel3_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',2))
segModel3_trans.compile(optimizer='adam',loss='Dice')
segModel4_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',3))
segModel4_trans.compile(optimizer='adam',loss='Dice')
segModel5_trans=keras.models.load_model('segModel_{}_{}.keras'.format('transaxial',4))
segModel5_trans.compile(optimizer='adam',loss='Dice')

direction=['coronal','sagittal','transaxial'][2]
folder_path='D:/img/rpe/RPE-SEGMENTIT'
files=os.listdir(folder_path)
dices_cor=[]
dices_sag=[]
dices_trans=[]
dices_2_5d=[]
pred_volumes_patient=[]
trans_volumes_patient=[]
real_volumes_patient=[]
for i in range(0,1):
    mask_path='{}/{}'.format(folder_path,files[i])
    mask=nib.load(mask_path)
    mask=mask.get_fdata()
    pathToDcmFolder='D:/img/rpe/anon_dixon/case{}'.format(files[i][0:4])
    img3d,voxelSize,iop=readDcmImage(pathToDcmFolder)
    predictedMask=np.zeros(img3d.shape)
    index=int(files[i][0:4])
    k=index % 5
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('coronal',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('coronal',k))
    for j in range(mask.shape[1]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[:,j,:]+=1
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('sagittal',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('sagittal',k))
    for j in range(mask.shape[0]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[j,:,:]+=1
    sliceIndexes=np.loadtxt('rcls_{}_{}_sliceIndexes.txt'.format('transaxial',k))
    patientIndexes_test=np.loadtxt('rcls_{}_{}_patientIndexes_test.txt'.format('transaxial',k))
    for j in range(mask.shape[2]):
        if sliceIndexes[patientIndexes_test==index][j]==1:
            predictedMask[:,:,j]+=1
    predictedMask=np.array(predictedMask>2.5,dtype=int)
    newMask_2_5d=np.zeros(predictedMask.shape)
    for d in ['coronal','sagittal','transaxial']:
        newMask=np.zeros(predictedMask.shape)
        denominators=np.zeros(predictedMask.shape)
        if np.max(predictedMask)>0:
            if d=='coronal':
                for j in range(mask.shape[1]):
                    if np.max(predictedMask[:,j,:])>0:
                        for i1 in range(0,mask.shape[0]-64,32):
                            if np.sum(predictedMask[i1:(i1+64),j,:])>min(1000,np.sum(predictedMask[:,j,:])/3):
                                slice=preprocessImgSlice(img3d[i1:(i1+64),j,:],64)
                                if k==0:
                                    pred=segModel1_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==1:
                                    pred=segModel2_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==2:
                                    pred=segModel3_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==3:
                                    pred=segModel4_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==4:
                                    pred=segModel5_cor.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                pred=cv2.resize(pred,(mask.shape[2],64))
                                newMask[i1:(i1+64),j,:]+=pred
                                denominators[i1:(i1+64),j,:]+=1
            if d=='sagittal':
                for j in range(mask.shape[0]):
                    if np.max(predictedMask[j,:,:])>0:
                        for i1 in range(0,mask.shape[1]-64,32):
                            if np.sum(predictedMask[j,i1:(i1+64),:])>min(1000,np.sum(predictedMask[:,j,:])/3):
                                slice=preprocessImgSlice(img3d[j,i1:(i1+64),:],64)
                                if k==0:
                                    pred=segModel1_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==1:
                                    pred=segModel2_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==2:
                                    pred=segModel3_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==3:
                                    pred=segModel4_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                if k==4:
                                    pred=segModel5_sag.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                pred=cv2.resize(pred,(mask.shape[2],64))
                                newMask[j,i1:(i1+64),:]+=pred
                                denominators[j,i1:(i1+64),:]+=1
            if d=='transaxial':
                for j in range(mask.shape[2]):
                    if np.max(predictedMask[:,:,j])>0:
                        for i1 in range(0,mask.shape[0]-64,32):
                            for j1 in range(0,mask.shape[0]-64,32):
                                if np.sum(predictedMask[i1:(i1+64),j1:(j1+64),j])>min(1000,np.sum(predictedMask[:,:,j])/3):
                                    slice=preprocessImgSlice(img3d[i1:(i1+64),j1:(j1+64),j],64)
                                    if k==0:
                                        pred=segModel1_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    if k==1:
                                        pred=segModel2_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    if k==2:
                                        pred=segModel3_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    if k==3:
                                        pred=segModel4_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    if k==4:
                                        pred=segModel5_trans.predict(np.array([slice]),verbose=0)[0][:,:,0]
                                    newMask[i1:(i1+64),j1:(j1+64),j]+=pred
                                    denominators[i1:(i1+64),j1:(j1+64),j]+=1
        denominators[denominators==0]=1
        newMask=newMask/denominators
        newMask=np.array(newMask>0.5,dtype=int)
        TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
        FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
        FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
        TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
        dice=2*TP/(2*TP+FN+FP)
        if d=='coronal':
            mask_cor=newMask
            dices_cor.append(dice)
        if d=='sagittal':
            mask_sag=newMask
            dices_sag.append(dice)
        if d=='transaxial':
            mask_trans=newMask
            dices_trans.append(dice)
            trans_volumes_patient.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(newMask))
        newMask_2_5d+=newMask
    newMask=np.array(newMask_2_5d>1.5,dtype=int)
    TP=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
    FN=np.sum(np.minimum(np.array(mask>=0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
    FP=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask>=0.5,dtype=int)))
    TN=np.sum(np.minimum(np.array(mask<0.5,dtype=int),np.array(newMask<0.5,dtype=int)))
    dice=2*TP/(2*TP+FN+FP)
    dices_2_5d.append(dice)
    print(i)
    pred_volumes_patient.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(newMask))
    real_volumes_patient.append(voxelSize[0]*voxelSize[1]*voxelSize[2]*np.sum(mask))

In [ ]:
#check patient's folder path
print(pathToDcmFolder)

In [ ]:
#save the result as nifti
mask=nib.load(mask_path)
header=mask.header
newNifti=nib.Nifti1Image(mask_sag, mask.affine, mask.header)
nib.save(newNifti,'predictedMask_patient0006_sagittalSegmentationModel.nii')
mask=mask.get_fdata()
print(mask.shape)

In [ ]:
#print the slice indexes showing the clear RPE segment most clearly
print(np.argmax(np.sum(np.sum(mask,axis=1),axis=1)))
print(np.argmax(np.sum(np.sum(mask,axis=0),axis=1)))
print(np.argmax(np.sum(np.sum(mask,axis=0),axis=0)))

In [ ]:
#check voxel size
print(voxelSize)

In [ ]:
#save an image from the sagittal direction
plt.imshow(cv2.resize(img3d[141,:,:],(288,round(40*4.5/0.69444442))),cmap='gray')
plot_outlines(cv2.resize(np.array(predictedMask[141,:,:],dtype='uint8'),(288,round(40*4.5/0.69444442))).T, lw=1, color='white')
plot_outlines(cv2.resize(np.array(mask[141,:,:],dtype='uint8'),(288,round(40*4.5/0.69444442))).T, lw=1, color='lightblue')
plot_outlines(cv2.resize(np.array(mask_trans[141,:,:],dtype='uint8'),(288,round(40*4.5/0.69444442))).T, lw=1, color='blue')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig_0002_sag.png',bbox_inches='tight')

In [ ]:
#save an image from the coronal direction
plt.imshow(cv2.resize(img3d[:,143,:],(288,round(40*4.41030884/0.69444442))),cmap='gray')
plot_outlines(cv2.resize(np.array(predictedMask[:,143,:],dtype='uint8'),(288,round(40*4.41030884/0.69444442))).T, lw=1, color='white')
plot_outlines(cv2.resize(np.array(mask[:,143,:],dtype='uint8'),(288,round(40*4.41030884/0.69444442))).T, lw=1, color='lightblue')
plot_outlines(cv2.resize(np.array(mask_trans[:,143,:],dtype='uint8'),(288,round(40*4.41030884/0.69444442))).T, lw=1, color='blue')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig_0001_cor.png',bbox_inches='tight')

In [ ]:
#save an image from the transaxial direction
plt.imshow(img3d[:,:,24],cmap='gray')
plot_outlines(predictedMask[:,:,24].T, lw=1, color='white')
plot_outlines(mask[:,:,24].T, lw=1, color='blue')
plot_outlines(mask_trans[:,:,24].T, lw=1, color='lightblue')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig_0014_trans.png',bbox_inches='tight')

In [ ]:
#plot and save pictures for the figures of the manuscript
kuv=np.ones((326,360))
for i in range(0,144):
    img_1=cv2.resize(np.array(img3d[:,2*(143-i),:],dtype='uint8'),(254,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+254),l:(l+288)]+=img_1/np.max(img_1)
kuv=kuv/np.max(kuv)
for i in range(74):
    for j in range(254+i,326):
        kuv[j,i]=1
for i in range(288,360):
    for j in range(0,i-288):
        kuv[j,i]=1
kuv[0:254,287:289]=1
kuv[253:255,0:288]=1
for i in range(2,74):
    kuv[(252+i):(253+i),(284+i):(287+i)]=1
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig1_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(0,145,24):
    img_1=cv2.resize(np.array(img3d[:,max(2*(143-i),5),:],dtype='uint8'),(254,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+254),l:(l+288)]=img_1/np.max(img_1)
    kuv[0:(l+254),(l+286):(l+288)]=1
    kuv[(l+252):(l+254),0:(l+288)]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig2_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(0,145,24):
    img_1=cv2.resize(np.array(img3d[max(2*(143-i),0),:,:],dtype='uint8'),(254,288))
    for i1 in range(72):
        for j1 in range(254):
            kuv[j1+i1,i1+2*i]=img_1[4*i1,j1]/np.max(img_1)
    kuv[0:254,(2*i):(2*i+2)]=1
    for i1 in range(72):
        kuv[(252+i1):(255+i1),2*i+i1]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig3_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(7):
    img_1=img3d[:,:,min(8*i,39)]
    for i1 in range(288):
        for j1 in range(72):
            kuv[j1+42*i,j1+i1]=img_1[i1,4*j1]/np.max(img_1)
    kuv[42*i:(42*i+2),0:288]=1
    for i1 in range(72):
        kuv[(42*i+i1):(42*i+i1+2),288+i1]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig4_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((432,144))
img_1=img3d[:,:,39]
for i1 in range(144):
    for j1 in range(288):
        kuv[j1+i1,i1]=img_1[2*i1,j1]/np.max(img_1)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig5_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.zeros((72,360))
for i in range(72):
    for j in range(i,72):
        kuv[j,i]=1
for i in range(288,360):
    for j in range(0,i-288):
        kuv[j,i]=1
for j in range(21,41):
    kuv[j,(82+j):(164+j)]=1
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig6_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.zeros((326,360))
for j in range(21,41):
    kuv[(j+127):(j+169),(82+j):(164+j)]=1
kuv[188:190,103:185]=0
kuv[148:190,183:185]=0
kuv[167:169,123:205]=0
kuv[167:210,123:125]=0
for i in range(20):
    kuv[(188+i):(190+i),(183+i):(185+i)]=0
    kuv[(148+i):(150+i),(103+i):(105+i)]=0
for i in range(74):
    for j in range(254+i,326):
        kuv[j,i]=1
for i in range(288,360):
    for j in range(0,i-288):
        kuv[j,i]=1
kuv[0:254,287:289]=1
kuv[253:255,0:288]=1
for i in range(2,74):
    kuv[(252+i):(253+i),(284+i):(287+i)]=1
kuv[72:326,70:72]=1
kuv[70:72,72:360]=1
for i in range(0,72):
    kuv[i:(i+2),i:(i+2)]=1
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig7_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(0,100,24):
    img_1=cv2.resize(np.array(img3d[:,140+round(i/24),:]/10,dtype='uint8'),(254,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+254),l:(l+288)]=img_1/np.max(img_1)
    kuv[0:(l+254),(l+286):(l+288)]=1
    kuv[(l+252):(l+254),0:(l+288)]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig8_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(0,100,24):
    img_1=cv2.resize(np.array(img3d[135+round(i/3),:,:]/10,dtype='uint8'),(254,288))
    for i1 in range(72):
        for j1 in range(254):
            kuv[j1+i1,i1+i]=img_1[4*i1,j1]/np.max(img_1)
    kuv[0:254,i:(i+2)]=1
    for i1 in range(72):
        kuv[(252+i1):(255+i1),i+i1]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig9_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(4):
    img_1=img3d[:,:,30+2*i]
    for i1 in range(288):
        for j1 in range(72):
            kuv[j1+42*i,j1+i1]=img_1[i1,4*j1]/np.max(img_1)
    kuv[42*i:(42*i+2),0:288]=1
    for i1 in range(72):
        kuv[(42*i+i1):(42*i+i1+2),288+i1]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig10_for_imgs.png',bbox_inches='tight')

In [ ]:
img_1=cv2.resize(np.array(img3d[:,143,:]/10,dtype='uint8'),(254,288))
img_1=np.transpose(img_1)
plt.imshow(img_1,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig11_for_imgs.png',bbox_inches='tight')

In [ ]:
img_1=cv2.resize(np.array(img3d[139,:,:]/10,dtype='uint8'),(254,288))
img_1=np.transpose(img_1)
plt.imshow(img_1,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig12_for_imgs.png',bbox_inches='tight')

In [ ]:
plt.imshow(np.transpose(img3d[:,:,31]),cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig13_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((360,360))
for i in range(0,150,24):
    img_1=cv2.resize(np.array(img3d[32:96,70+round(i/6),:]/10,dtype='uint8'),(288,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+288),l:(l+288)]=img_1/np.max(img_1)
    kuv[0:(l+288),(l+286):(l+288)]=1
    kuv[(l+286):(l+288),0:(l+288)]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig14_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((360,360))
for i in range(0,130,24):
    img_1=cv2.resize(np.array(img3d[125+round(i/3),96:160,:]/10,dtype='uint8'),(288,288))
    for i1 in range(72):
        for j1 in range(288):
            kuv[j1+i1,i1+i]=img_1[4*i1,j1]/np.max(img_1)
    kuv[0:288,i:(i+2)]=1
    for i1 in range(72):
        kuv[(286+i1):(288+i1),i+i1]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig15_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((360,360))
for i in range(7):
    img_1=cv2.resize(np.array(img3d[96:160,96:160,25+2*i]/10,dtype='uint8'),(288,288))
    for i1 in range(288):
        for j1 in range(72):
            kuv[j1+42*i,j1+i1]=img_1[i1,4*j1]/np.max(img_1)
    kuv[42*i:(42*i+2),0:288]=1
    for i1 in range(72):
        kuv[(42*i+i1):(42*i+i1+2),288+i1]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig16_for_imgs.png',bbox_inches='tight')

In [ ]:
print(np.argmax(np.sum(np.sum(mask_trans,axis=1),axis=1)))
print(np.argmax(np.sum(np.sum(mask_trans,axis=0),axis=1)))
print(np.argmax(np.sum(np.sum(mask_trans,axis=0),axis=0)))

In [ ]:
kuv=np.ones((360,360))
for i in range(0,150,24):
    img_1=cv2.resize(np.array(mask_cor[96:160,100,:],dtype='uint8'),(288,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+288),l:(l+288)]=img_1
    kuv[0:(l+288),(l+286):(l+288)]=1
    kuv[(l+286):(l+288),0:(l+288)]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig17_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((360,360))
for i in range(0,130,24):
    img_1=cv2.resize(np.array(mask_sag[129,96:160,:],dtype='uint8'),(288,288))
    for i1 in range(72):
        for j1 in range(288):
            kuv[j1+i1,i1+i]=img_1[4*i1,j1]
    kuv[0:288,i:(i+2)]=1
    for i1 in range(72):
        kuv[(286+i1):(288+i1),i+i1]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig18_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((360,360))
for i in range(7):
    img_1=cv2.resize(np.array(mask_trans[96:160,96:160,20+i],dtype='uint8'),(288,288))
    for i1 in range(288):
        for j1 in range(72):
            kuv[j1+42*i,j1+i1]=img_1[i1,4*j1]
    kuv[42*i:(42*i+2),0:288]=1
    for i1 in range(72):
        kuv[(42*i+i1):(42*i+i1+2),288+i1]=1
kuv=kuv/np.max(kuv)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig19_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(0,144):
    img_1=cv2.resize(np.array(mask_cor[:,2*(143-i),:],dtype='uint8'),(254,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+254),l:(l+288)]+=img_1
kuv=np.array(kuv>1.3,dtype=int)
for i in range(74):
    for j in range(254+i,326):
        kuv[j,i]=1
for i in range(288,360):
    for j in range(0,i-288):
        kuv[j,i]=1
kuv[0:254,287:289]=1
kuv[253:255,0:288]=1
for i in range(2,74):
    kuv[(252+i):(253+i),(284+i):(287+i)]=1
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig20_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(0,144):
    img_1=cv2.resize(np.array(mask_sag[:,2*(143-i),:],dtype='uint8'),(254,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+254),l:(l+288)]+=img_1
kuv=np.array(kuv>1.3,dtype=int)
for i in range(74):
    for j in range(254+i,326):
        kuv[j,i]=1
for i in range(288,360):
    for j in range(0,i-288):
        kuv[j,i]=1
kuv[0:254,287:289]=1
kuv[253:255,0:288]=1
for i in range(2,74):
    kuv[(252+i):(253+i),(284+i):(287+i)]=1
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig21_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(0,144):
    img_1=cv2.resize(np.array(mask_trans[:,2*(143-i),:],dtype='uint8'),(254,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+254),l:(l+288)]+=img_1
kuv=np.array(kuv>1.3,dtype=int)
for i in range(74):
    for j in range(254+i,326):
        kuv[j,i]=1
for i in range(288,360):
    for j in range(0,i-288):
        kuv[j,i]=1
kuv[0:254,287:289]=1
kuv[253:255,0:288]=1
for i in range(2,74):
    kuv[(252+i):(253+i),(284+i):(287+i)]=1
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig22_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((326,360))
for i in range(0,144):
    img_1=cv2.resize(np.array(newMask[:,2*(143-i),:],dtype='uint8'),(254,288))
    img_1=np.transpose(img_1)
    l=round((143-i)/2)
    kuv[l:(l+254),l:(l+288)]+=img_1
kuv=np.array(kuv>1.3,dtype=int)
for i in range(74):
    for j in range(254+i,326):
        kuv[j,i]=1
for i in range(288,360):
    for j in range(0,i-288):
        kuv[j,i]=1
kuv[0:254,287:289]=1
kuv[253:255,0:288]=1
for i in range(2,74):
    kuv[(252+i):(253+i),(284+i):(287+i)]=1
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig23_for_imgs.png',bbox_inches='tight')

In [ ]:
kuv=np.ones((432,144))
img_1=cv2.resize(np.array(img3d[96:160,96:160,31]/10,dtype='uint8'),(288,288))
for i1 in range(144):
    for j1 in range(288):
        kuv[j1+i1,i1]=img_1[2*i1,j1]/np.max(img_1)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig24_for_imgs.png',bbox_inches='tight',pad_inches=0)

In [ ]:
kuv=np.ones((432,144))
img_1=cv2.resize(np.array(mask_trans[96:160,96:160,31],dtype='uint8'),(288,288))
for i1 in range(144):
    for j1 in range(288):
        kuv[j1+i1,i1]=img_1[2*i1,j1]/np.max(img_1)
plt.imshow(kuv,cmap='gray',origin='lower')
plt.axis('off')
fig=plt.gcf()
fig.savefig('fig25_for_imgs.png',bbox_inches='tight',pad_inches=0)

In [ ]:
#print the model architecture of the used U-Net (to check number of parameters)
inputs = tf.keras.layers.Input(shape=(64,64,1))

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)

#Expansive path
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c3)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)#or linear

segModel = tf.keras.Model(inputs=inputs, outputs=outputs)

print(segModel.summary())